# Prepare dataset

In this notebook, we will prepare the source csv data for use later.
At last we will store the modified dataframe into a new csv.

> Note: we will use a public Japanese company name as the data source. You can download [here](https://info.gbiz.go.jp/hojin/DownloadTop).


In [14]:
# Let's install the dependencies..
!pip install numpy pandas -q

/Users/leung.tsz.kit/Desktop/work/code/test/github_repos/ann_search_sample/nearest_neighbor/.venv/bin/pip
zsh:1: /Users/leung.tsz.kit/Desktop/work/code/test/github_repos/ann_search_sample/nearest_neighbor/.venv/bin/pip: bad interpreter: /Users/leung.tsz.kit/Desktop/work/code/test/github_repos/ann_search_sample/annoy/.venv/bin/python3.9: no such file or directory


In [5]:
import numpy as np
import pandas as pd
import os
import unicodedata
import time
from pprint import pprint as pp

Declare some variables...

In [3]:

# Point to the csv you download
csv_path = "./tmp/info_gbiz/Kihonjoho_UTF-8.csv"

## Load the dataset

We will load the dataset and clean the text.

In [4]:
df = pd.read_csv(csv_path, dtype=str)
# we only need these fields:
df = df[["法人名", "郵便番号", "本社所在地"]]
df.sample(5)

,法人名,郵便番号,本社所在地
4979406,オシアナス株式会社,1040055,東京都中央区豊海町４－１８東京水産ビル５階
3233119,永田纎維株式会社,8300023,福岡県久留米市中央町２４番地の１
4940059,田川ＤＸ推進室株式会社,8250002,福岡県田川市大字伊田２６２１番１号ＪＲ田川伊田駅舎
4051398,重車輛整備株式会社,6500046,兵庫県神戸市中央区港島中町１丁目１番１８号
1798139,有限会社泰成企画,2130013,神奈川県川崎市高津区末長１１１６番地６


If you browse through more samples, you will notice some problems:
- Some words are in full-width but some are in half-width.
- We want the postal code in this format: xxx-xxxx
- Replace NaN value by empty string

First we prepare a function to normalize text.

In [8]:
# This is a common step for preprocessing CJK characters.
def normalize_text(text):
    text = unicodedata.normalize('NFKC', text)#.encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

# For example
normalize_text('ｱア福岡県田川市大字伊田２６２１番１号ＪＲ田川伊田駅舎')

'アア福岡県田川市大字伊田2621番1号JR田川伊田駅舎'

Then, we convert the dataset by several operation.

In [9]:
# fill Na with empty string
df = df.fillna("")
# Normalize text
df = df.applymap(normalize_text)
# For 郵便番号, we want them in this format xxx-xxxx, if it is 7-digit
def add_hyphen(postal_code):
    if len(postal_code) == 7:
        return postal_code[:3] + '-' + postal_code[3:]
    else:
        return postal_code
df["郵便番号"] = df["郵便番号"].apply(add_hyphen)
df.sample(5)


,法人名,郵便番号,本社所在地
457911,医療法人志賀クリニック,327-0104,栃木県佐野市赤見町3152番地
1607549,株式会社Viels,221-0005,神奈川県横浜市神奈川区松見町3丁目492番地14
1148229,有限会社木内シール印刷,115-0055,東京都北区赤羽西3丁目28番15号
229330,株式会社ゼン,980-0802,宮城県仙台市青葉区二日町2番22号
5130357,株式会社ホテルいずくら,994-0025,山形県天童市鎌田本町3丁目2番26号


Almost there! We want a concatenated field containing the text columns:

In [10]:
df["concat_name"] = df["法人名"] + " " + df["郵便番号"] + " " + df["本社所在地"]
# Remove any leading/trailing space
df = df.applymap(str.strip)
df.sample(5)

,法人名,郵便番号,本社所在地,concat_name
718091,株式会社キヨシゲ,279-0025,千葉県浦安市鉄鋼通り2丁目4番3号,株式会社キヨシゲ 279-0025 千葉県浦安市鉄鋼通り2丁目4番3号
456247,有限会社たなか,323-0032,栃木県小山市天神町1丁目2番7号,有限会社たなか 323-0032 栃木県小山市天神町1丁目2番7号
3849864,瀬田商事株式会社,171-0021,東京都豊島区西池袋5丁目5番21-1608号,瀬田商事株式会社 171-0021 東京都豊島区西池袋5丁目5番21-1608号
2071549,株式会社ツカサ,504-0963,岐阜県各務原市那加北栄町28番地の1,株式会社ツカサ 504-0963 岐阜県各務原市那加北栄町28番地の1
225414,有限会社REAL CRAFT,983-0034,宮城県仙台市宮城野区扇町7丁目1番26号,有限会社REAL CRAFT 983-0034 宮城県仙台市宮城野区扇町7丁目1番26号


Run below cell if you want to take a subset only.

In [9]:
# data = data.sample(1000)

At last, we will store the new dataframe for use later

In [11]:
df.to_csv("./tmp/converted_company_ds.csv", index=False)